In [14]:
import cv2
import numpy as np
import cv2.aruco as aruco
import os
from time import sleep

# Print video capture devices
def print_video_devices():
    for i in range(10):
        cap = cv2.VideoCapture(i)
        if cap.read()[0]:
            print(f"Device {i} is a camera")
        else:
            print(f"Device {i} is not a camera")
        cap.release()

print_video_devices()


Device 0 is not a camera
Device 1 is a camera
Device 2 is a camera
Device 3 is not a camera
Device 4 is not a camera
Device 5 is not a camera
Device 6 is not a camera
Device 7 is not a camera
Device 8 is not a camera
Device 9 is not a camera


In [ ]:
# Generate aruco markers
def generate_aruco_markers():
    # Dictionary of 250 markers
    aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)

    # Create an image from the dictionary
    for i in range(250):
        img = aruco.drawMarker(aruco_dict, i, 700)

        # Get current dir path
        path = os.getcwd()

        cv2.imwrite(path + "/markers/" + str(i) + ".jpg", img)


# Detect aruco markers
def detect_aruco_markers(aruco_dict, frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect aruco markers
    parameters = aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Draw detected markers
    frame = aruco.drawDetectedMarkers(frame, corners, ids)

    return frame, corners, ids


In [17]:
# Take a picture each second

#cam = cv2.VideoCapture(1)
#for i in range(20):
#
#    ret, frame = cam.read()
#
#    print(f'Taking picture {i}')
#        
#    cv2.imwrite("C:\\Users\\Marcus\\Documents\\GitHub\\Zephyr-drone\\Python\\calibration\\" + str(i) + ".jpg", frame)
#    sleep(1)
    

Taking picture 0
Taking picture 1
Taking picture 2
Taking picture 3
Taking picture 4
Taking picture 5
Taking picture 6
Taking picture 7
Taking picture 8
Taking picture 9
Taking picture 10
Taking picture 11
Taking picture 12
Taking picture 13
Taking picture 14
Taking picture 15
Taking picture 16
Taking picture 17
Taking picture 18
Taking picture 19


In [21]:
# Calibrate camera
def calibrate_camera():
    # Get current dir path
    path = os.getcwd()

    # Termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6 * 7, 3), np.float32)

    # Create an array with coordinates of the corners
    objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane

    # Get images from the calibration folder
    images = [f for f in os.listdir(path + "/calibration") if f.endswith('.jpg')]

    for fname in images:
        # Read image
        img = cv2.imread(path + "/calibration/" + fname)

        # Convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)

            # Refine the corners
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

            # Add image points
            imgpoints.append(corners2)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (7, 6), corners2, ret)

    # Calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return ret, mtx, dist, rvecs, tvecs

# Undistort image
def undistort_image(img, mtx, dist):
    # Undistort image
    dst = cv2.undistort(img, mtx, dist, None, mtx)

    return dst

ret, mtx, dist, rvecs, tvecs = calibrate_camera()

dst = cv2.imread("C:\\Users\\Marcus\\Documents\\GitHub\\Zephyr-drone\\Python\\calibration\\0.jpg")

img = undistort_image(dst, mtx, dist)

# Set img side by side with undistorted image
img = np.hstack((img, dst))
cv2.imshow('img', img)
cv2.waitKey(0)

-1

In [2]:
cam = cv2.VideoCapture(1)

# Show camera feed
while True:
    ret, frame = cam.read()
    img,corners,ids = detect_aruco_markers(aruco.Dictionary_get(aruco.DICT_6X6_250), frame)
    cv2.imshow("Camera", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
